In [32]:

(import "IPython.display")
(define SVG IPython.display.SVG)

(import "builtins")
(define python builtins)

(import "math")

(math)

## Data Abstration

In [3]:

(define (attach-tag type-tag contents)
  (cons type-tag contents))

(define (type-tag value)
  (car value))

(define (contents value)
  (cdr value))

In [20]:


(define (make-shape tag . args)
    (attach-tag tag args))

(define (circle r) 
  (make-shape 'circle r))

(define (rectangle w h) 
  (make-shape 'rectangle w h))

(define (make-transformation tag . args)
    (attach-tag tag args))

(define (rotate angle)
  (make-transformation 'rotate angle))

(define (translate x y)
  (make-transformation 'translate x y))

(define (scale x y)
  (make-transformation 'scale x y))



(define (apply-transformation t s)
  (make-shape 'T t s))

(define (identity)
  (make-transformation 'I))

(define (T0)
  (make-transform 'T0))

(define (combine shapes)
  (make-shape 'S+ shapes))

(define (compose-transformations transforms)
  (make-transformation 'T* transforms))

(define (add-transformations transforms)
  (make-transformation 'T+ transforms))

(define (T* . transformations)
  (compose-transformations transformations))



**Problem:** Can you implement the following:

* repeat 
* beside
* below
* quartlet
* cycle4



In [12]:
(define (repeat t n) 
      (cond ((= n 0) (lambda (x) x))
            (else 
                 (add-transformations 
                      t 
                      (repeat (compose-transformations (list t t)) (- n 1))))))

In [25]:
(define (beside s1 s2) 
      (combine
           (list (apply-transformation
                (T*
                     (scale 0.5 1)
                     (translate -50 0))
                s1)
           (apply-transformation
                (T*
                     (scale 0.5 1)
                     (translate 50 0))
                s2))))

In [26]:
(beside (circle 100) (circle 100))

(S+ ((T (T* ((scale 0.5 1) (translate -50 0))) (circle 100)) (T (T* ((scale 0.5 1) (translate 50 0))) (circle 100))))

In [19]:
(define (below s1 s2) 
      (combine
           (apply-transformation
                (compose-transformations
                     (scale 1 0.5)
                     (translate 0 -50))
                s1)
           (apply-transformation
                (compose-transformations
                     (scale 1 0.5)
                     (translate 0 50))
                s2)))

In [15]:
(define (quartlet p q r s) 
      (below (beside p q) (beside r s)))

In [16]:
(define (cycle-4) 
      (lambda (x) (quartlet x x x x)))

### Rendering : Approach 1

In [34]:
(define proc-table (python.dict))



(define (topy value)
  (if (list? value) 
      (python.tuple (map topy value))
      value))

(define (key op types)
  (topy (list op types)))

(define (put op type value)
  (proc-table.__setitem__ 
   (key op type)
   value))



(define (get op type)
  (proc-table.__getitem__ 
   (key op type)))



In [35]:
(define (render-node shape)
  (dispatch 'render shape))

;; (define (add-shapes s1 s2)
;;   (dispatch 'add s1 s2))

(define (render-circle c)
  (format "<circle r='~a' />" (car c)))

(define (render-rectangle r)
  (let ((w (car r))
        (h (cadr r)))
  (format "<rect x='~a' y='~a' width='~a' height='~a' />" 
          (/ w -2.0)
          (/ h -2.0)
          w
          h)))


(put 'render '(circle) render-circle)
(put 'render '(rectangle) render-rectangle)

In [64]:
(define (dispatch op . args)
    (let ((type-tags (map type-tag args)))
      (python.print op type-tags)
      (let ((proc (get op type-tags)))
        (python.print (map contents args))
        (apply proc (map contents args)))))

In [65]:


(render-node (circle 50))

render (circle)
((50))


"<circle r='50' />"

In [63]:
(render-node (rectangle 100 50))

render (rectangle)
((100 50))


("<rect x='-50.0' y='-25.0' width='100' height='50' />")


## Assignment and Local State


In [69]:


(define (make-withdraw balance)
  (lambda (amount)
    (begin
     (set! balance (- balance amount))
     balance)))

(define w1 (make-withdraw 100))
(define w2 (make-withdraw 50))



(w1 20)
(w1 20)

60



Problem: Write a procedure make-accumulator that returns a an accumulator. Accumulator is a procedure thats a number as argument and accumulates it to a sum.
```
(define a1 (make-accumulator 10))

(a1 5)
15

(a1 2)
17

```

In [70]:
(define (make-accumulator initial) 
  (lambda (amount) 
    (begin 
         (set! initial (+ initial amount))
         initial)))

In [74]:
(define a1 (make-accumulator 10))

In [75]:
(a1 2)
(a1 2)

14

In [84]:
(define (make-account balance) 
  (define (withdraw amount) 
    (begin
     (set! balance (- balance amount))
     balance))
  (define (deposit amount) 
    (begin
     (set! balance (+ balance amount))
     balance))
  (define (dispatch method) 
    (cond ((eq? method 'withdraw) withdraw)
          ((eq? method 'deposit) deposit)
          ((eq? method 'balance) balance)
          (else (error "wrong method " method))))
  dispatch)

In [85]:
(define a1 (make-account 100))
(define a2 (make-account 50))

In [86]:
((a1 'withdraw) 10)

90

In [87]:
(a1 'balance)

90

In [79]:
((a1 'withdraw) 10)

80

In [80]:
((a2 'withdraw) 20)

30

In [81]:
(define (withdraw account amount) 
  ((account 'withdraw) amount))

(define (deposit account amount) 
  ((account 'deposit) amount))

In [82]:
(deposit a1 20)

100

In [88]:
(- 1 2 3)

-4